INSTALL
- FAISS - 
- Sent2Vec

In [1]:
import numpy as np
import faiss   

In [2]:
import os
import time
name = "TED-sent2vec-"+ str(time.strftime("%Y%m%d-%H%M"))

!mkdir $name
path = os.getcwd() + "/" + name
os.chdir(path)

!pwd


os.chdir(path)


# import os
# os.chdir("/home/lw/clc_fairseq/TED-20210121-1335")

/home/chinh/clc_data/dataset/TED-sent2vec-20210410-1431


In [3]:
!wget https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
!mkdir data_bin
!tar -xvf  'en_vi_iwslt.tar.gz' -C 'data_bin'

--2021-04-10 14:31:56--  https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz [following]
--2021-04-10 14:31:57--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7497739 (7,2M) [application/octet-stream]
Saving to: ‘en_vi_iwslt.tar.gz’

en_vi_iwslt.tar.gz  100%[===================>]   7,15M  2,86MB/s    in 2,5s    

2021-04-10 14:32:01 (2,86 MB/s) - ‘en_vi_iwslt.tar.gz’ saved [7497739/7497

In [4]:
# import file
import numpy as np
import glob

en_files = sorted(glob.glob("data_bin/*.en"))
vi_files = sorted(glob.glob("data_bin/*.vi"))
print(en_files)
print(vi_files)

lst_en = []
lst_vi = []

start = time.time()

for file in en_files:
  with open(file) as file_in:
    for line in file_in:
      lst_en.append(line)

for file in vi_files:
  with open(file) as file_in:
    for line in file_in:
      lst_vi.append(line)

print(len(lst_en))
print(len(lst_vi))

['data_bin/TED.en']
['data_bin/TED.vi']
103624
103624


In [5]:
# SPLIT TRAIN, TEST, VALID
from sklearn.model_selection import train_test_split
en_train, en_test_valid, vi_train, vi_test_valid =  train_test_split(lst_en, lst_vi, test_size=0.2, random_state=123)
en_valid, en_test, vi_valid, vi_test = train_test_split(en_test_valid, vi_test_valid, test_size=0.5, random_state=123)
print(len(en_train))
print(len(vi_train))
print(len(en_valid))
print(len(vi_valid))
print(len(en_test))
print(len(vi_test))
print(en_train[20])
print(vi_train[20])
print(en_valid[-1])
print(vi_valid[-1])
print(en_test[-1])
print(vi_test[-1])

82899
82899
10362
10362
10363
10363
And it's based on pizza .

Và nó dựa trên pizza .

You formulate a recipe based upon the factors you can control , whether it be the skills or the fertilizer or the plant choice .

Bạn xây dựng một công thức dựa trên những yếu tố có thể kiểm soát được , đó có thể là các kỹ năng hay phân bón hay chọn giống .

All of these are parts of his life and what he remembers .

Những điều này là một phần cuộc đời ông và là những gì ông còn nhớ



In [6]:
file_variable = [en_train,vi_train,en_valid,vi_valid,en_test,vi_test]
file_names = ["en_train","vi_train","en_valid","vi_valid","en_test","vi_test"]

for i in range(len(file_names)):
  with open(file_names[i], 'w') as f:
    for item in file_variable[i]:
      f.write(item)
!ls -al

total 30156
drwxrwxr-x 3 chinh chinh     4096 Thg 4  10 14:32 .
drwxrwxr-x 4 chinh chinh     4096 Thg 4  10 14:31 ..
drwxrwxr-x 2 chinh chinh     4096 Thg 4  10 14:32 data_bin
-rw-rw-r-- 1 chinh chinh   996149 Thg 4  10 14:32 en_test
-rw-rw-r-- 1 chinh chinh  8024744 Thg 4  10 14:32 en_train
-rw-rw-r-- 1 chinh chinh  1000856 Thg 4  10 14:32 en_valid
-rw-rw-r-- 1 chinh chinh  7497739 Thg 4  10 14:32 en_vi_iwslt.tar.gz
-rw-rw-r-- 1 chinh chinh  1327417 Thg 4  10 14:32 vi_test
-rw-rw-r-- 1 chinh chinh 10671354 Thg 4  10 14:32 vi_train
-rw-rw-r-- 1 chinh chinh  1330789 Thg 4  10 14:32 vi_valid


In [7]:
start = time.time()
import sent2vec
model = sent2vec.Sent2vecModel()
model.load_model('../../../wiki_unigrams.bin')

print('=== Ending, Total time (second): ', time.time() - start)

=== Ending, Total time (second):  9.121287107467651


In [8]:
start = time.time()
srcfile = "en_train"
tarfile = "vi_train"
lst_srcfile = en_train
lst_tarfile = vi_train

vec_lst_srcfile = model.embed_sentences(lst_srcfile)
print('=== Ending, Total time (second): ', time.time() - start)

=== Ending, Total time (second):  0.48729658126831055


# Calculate similarity

In [9]:
import faiss
vec_lst_srcfile_copy = vec_lst_srcfile.copy()
start = time.time()
index = faiss.IndexFlatIP(vec_lst_srcfile_copy.shape[1])
index.ntotal
faiss.normalize_L2(vec_lst_srcfile_copy)
index.add(vec_lst_srcfile_copy)
k = 6
distance, index = index.search(vec_lst_srcfile_copy, k)

print('=== Ending, Total time (second): ', time.time() - start)
print(len(distance))
print(len(index))

print(index)
print(distance)

=== Ending, Total time (second):  20.9006609916687
82899
82899
[[    0 54197  5058 28074 35053  4852]
 [    1 69584 46276 27557 60258 72690]
 [    2 47179 39288 49898 44045 34365]
 ...
 [82896  2436 43083 79248 36194 13450]
 [82897 53187 58246 37537 56012 47320]
 [82898 30329 35171 65086 82586 14252]]
[[1.         0.78319913 0.78007334 0.7730269  0.772054   0.772054  ]
 [0.99999994 0.5116507  0.50914973 0.5079074  0.4961168  0.4923037 ]
 [1.0000001  0.56854033 0.5668709  0.56416297 0.55664337 0.55497247]
 ...
 [0.99999994 0.56561005 0.56018656 0.5581732  0.5439784  0.5426278 ]
 [1.0000002  0.5442871  0.54330283 0.5377911  0.5229071  0.508623  ]
 [1.         0.7300915  0.7274866  0.7253362  0.72123575 0.7167394 ]]


In [10]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance as dt

count = 0
for i in range(len(lst_srcfile)):
    j = 1 #1 is index of biggest simmilarity
    score = distance[i][j]
    a = vec_lst_srcfile[i]
    b = vec_lst_srcfile[index[i][j]]
    
    dot_product = np.dot(a, b) # x.y
    norm_a = np.linalg.norm(a) #|x|
    norm_b = np.linalg.norm(b) #|y|
    sim_res = dot_product / (norm_a * norm_b)

    if score >=0.98 :
        count+=1
        print("==========================")
        for m in range(len(index[i])):
            print("--------", distance[i][m])
        print(i, index[i][j], distance[i][j], sim_res, 1-dt.cosine(a,b))
        print(lst_srcfile[i])
        print(lst_srcfile[index[i][j]])
        print("==========================")
print(count)

-------- 1.0000001
-------- 1.0000001
-------- 0.97406906
-------- 0.921575
-------- 0.8895616
-------- 0.88824534
188 188 1.0000001 0.9999999 1.0
Well let me show you .

Well let me show you .

-------- 1.0000005
-------- 1.0000005
-------- 1.0000005
-------- 1.0000005
-------- 1.0000005
-------- 1.0000005
215 4257 1.0000005 1.0 1.0
Bill's hasn't .

So that's Paris .

-------- 0.9999998
-------- 0.9999998
-------- 0.83674026
-------- 0.81895894
-------- 0.818376
-------- 0.8106055
230 16932 0.9999998 1.0 1.0
I need that data in November .

I need that data in September .

-------- 0.9999999
-------- 0.9999999
-------- 0.9999999
-------- 0.9999999
-------- 0.9531639
-------- 0.9415051
399 57817 0.9999999 0.9999999 1.0
And that's how it works .

That's how it works .

-------- 1.0
-------- 0.9942273
-------- 0.90641814
-------- 0.8886764
-------- 0.88847697
-------- 0.85590494
452 74354 0.9942273 0.9942271 0.9942271113395691
So , we started Parikrma .

So we started .

-------- 1.000000

-------- 1.0000001
-------- 0.9948911
-------- 0.88026774
-------- 0.8674534
-------- 0.8608908
-------- 0.8565068
8100 22628 0.9948911 0.9948909 0.9948908686637878
And I went away .

Happy , away I went .

-------- 1.0000001
-------- 1.0000001
-------- 1.0000001
-------- 1.0000001
-------- 1.0000001
-------- 1.0000001
8147 8147 1.0000001 1.0 1.0
JE : Thank you , Tony .

JE : Thank you , Tony .

-------- 0.99999976
-------- 0.99999976
-------- 0.8856574
-------- 0.8806661
-------- 0.8806661
-------- 0.8806661
8171 8171 0.99999976 0.99999994 1.0
And Nokia as well .

And Nokia as well .

-------- 1.0
-------- 0.99286044
-------- 0.94237435
-------- 0.9197324
-------- 0.9017957
-------- 0.8882394
8182 67251 0.99286044 0.9928605 0.9928605556488037
One more .

Wait , there's more .

-------- 1.0
-------- 1.0
-------- 0.850147
-------- 0.76373553
-------- 0.7363682
-------- 0.717831
8184 35758 1.0 1.0 1.0
T : I want to change my life !

S : I want to change my life !

-------- 0.9999998
----

-------- 0.99999964
-------- 0.99999964
-------- 0.99999964
-------- 0.99999964
-------- 0.99999964
-------- 0.99999964
15723 5780 0.99999964 0.99999994 1.0
Laura Snyder : The Philosophical Breakfast Club

Nora York : Singing " What I Want "

-------- 1.0000001
-------- 0.98524487
-------- 0.92191625
-------- 0.90103483
-------- 0.90103483
-------- 0.8971478
15777 29268 0.98524487 0.98524487 0.9852449297904968
Wild is love .

And the L is love .

-------- 1.0
-------- 1.0
-------- 1.0
-------- 1.0
-------- 1.0
-------- 1.0
15977 12185 1.0 1.0 1.0
Death : Adam ?

Sergey Brin : Why Google Glass ?

-------- 1.0000001
-------- 1.0000001
-------- 0.9917237
-------- 0.9917237
-------- 0.9485178
-------- 0.9485178
16249 6247 1.0000001 1.0 1.0
But it isn't .

But it doesn't .

-------- 0.99999994
-------- 0.99999994
-------- 0.9927251
-------- 0.95838875
-------- 0.9580286
-------- 0.94959176
16351 16351 0.99999994 1.0000001 1.0
That won't do .

That won't do .

-------- 1.0
-------- 1.0
-----

-------- 1.0000001
-------- 1.0000001
-------- 1.0000001
-------- 1.0000001
-------- 1.0000001
-------- 1.0000001
23788 8147 1.0000001 1.0 1.0
Larry Page : Thank you , Sergey .

JE : Thank you , Tony .

-------- 0.9999997
-------- 0.9999997
-------- 0.9999997
-------- 0.901966
-------- 0.8907714
-------- 0.87209165
23836 82531 0.9999997 1.0000001 1.0
Let me tell you about Beatrice Biira .

Let me tell you about Saima .

-------- 1.0000004
-------- 1.0000004
-------- 1.0000004
-------- 1.0000004
-------- 1.0000004
-------- 1.0000004
23848 3833 1.0000004 1.0000001 1.0
Thank you , TED .

Thank you , Chris .

-------- 0.9999998
-------- 0.9851829
-------- 0.9829706
-------- 0.9553591
-------- 0.9528507
-------- 0.9502096
23882 25658 0.9851829 0.985183 0.9851830005645752
So , then what do you do ?

So , what do you do ?

-------- 0.9999998
-------- 0.9999998
-------- 0.9468329
-------- 0.9077494
-------- 0.9017922
-------- 0.9017921
24137 24137 0.9999998 0.99999994 1.0
She left .

She left 

-------- 1.0
-------- 1.0
-------- 1.0
-------- 1.0
-------- 1.0
-------- 0.92273533
30624 48998 1.0 1.0 1.0
So , yeah .

Um , yeah .

-------- 1.0
-------- 1.0
-------- 0.9959799
-------- 0.952077
-------- 0.94304436
-------- 0.9322227
30744 30744 1.0 1.0 1.0
And you know what ?

And you know what ?

-------- 0.99999946
-------- 0.99999946
-------- 0.953933
-------- 0.9096023
-------- 0.8847149
-------- 0.8847149
30831 30831 0.99999946 1.0 1.0
And it's pretty simple .

And it's pretty simple .

-------- 1.0000001
-------- 1.0000001
-------- 0.9724817
-------- 0.95158064
-------- 0.90909976
-------- 0.9068519
30849 64488 1.0000001 1.0000001 1.0
and Albania and Tunisia .

That's Sage and Annalisa and Rider .

-------- 1.0000001
-------- 1.0000001
-------- 0.9688151
-------- 0.9688151
-------- 0.9653522
-------- 0.9653521
30863 79514 1.0000001 1.0000001 1.0
This is a SOLE in Newcastle .

This is a Telecentro in Sao Paulo .

-------- 1.0
-------- 0.9915875
-------- 0.98173326
-------- 0.9

...

-------- 1.0000002
-------- 0.993392
-------- 0.9337294
-------- 0.9337294
-------- 0.91923034
-------- 0.91535485
47131 5334 0.993392 0.993392 0.9933919310569763
So we have The Echo Park Time Travel Mart .

Consequently , we have .

-------- 1.0000001
-------- 1.0000001
-------- 0.9301737
-------- 0.9238174
-------- 0.9238174
-------- 0.9219663
47147 45033 1.0000001 1.0000001 1.0
Why aren't you ?

Will you ?

-------- 0.99999976
-------- 0.99999976
-------- 0.8736666
-------- 0.8318661
-------- 0.817668
-------- 0.817668
47238 41875 0.99999976 1.0 1.0
They found something called Facebook .

They found something called Twitter .

-------- 0.99999994
-------- 0.99999994
-------- 0.99999994
-------- 0.99999994
-------- 0.99999994
-------- 0.99999994
47366 5104 0.99999994 1.0 1.0
RG : Cruel .

DP : " English Country Garden . "

-------- 0.99999994
-------- 0.98168147
-------- 0.8952989
-------- 0.8938517
-------- 0.8936355
-------- 0.8797838
47403 25309 0.98168147 0.9816816 0.981681644916

-------- 0.9999999
-------- 0.9954816
-------- 0.93575346
-------- 0.93575346
-------- 0.88472086
-------- 0.8764794
54857 59022 0.9954816 0.9954818 0.9954818487167358
Hey , why not ?

And why not ?

-------- 0.99999976
-------- 0.98318064
-------- 0.9670424
-------- 0.9626933
-------- 0.9626933
-------- 0.9597681
54919 10103 0.98318064 0.98318064 0.9831806421279907
Or look at Google .

Now look at the Moon .

-------- 0.99999994
-------- 0.99999994
-------- 0.992003
-------- 0.992003
-------- 0.992003
-------- 0.96813315
54933 54933 0.99999994 1.0 1.0
Chimborazo !

Chimborazo !

-------- 0.9999997
-------- 0.9999997
-------- 0.9999997
-------- 0.9999997
-------- 0.9999997
-------- 0.9999997
54963 47491 0.9999997 1.0 1.0
This is Evangelina Etienne from Massachusetts .

This is Allan Okrainec from Toronto .

-------- 0.99999994
-------- 0.98181593
-------- 0.98181593
-------- 0.95204973
-------- 0.9324438
-------- 0.9324438
54979 50082 0.98181593 0.98181605 0.9818159937858582
It was cal

-------- 1.0000001
-------- 1.0000001
-------- 1.0000001
-------- 1.0000001
-------- 1.0000001
-------- 1.0000001
62517 8147 1.0000001 1.0 1.0
PM : Thank you , Sheryl .

JE : Thank you , Tony .

-------- 0.99999994
-------- 0.99999994
-------- 0.99999994
-------- 0.99999994
-------- 0.99999994
-------- 0.99999994
62599 5104 0.99999994 1.0 1.0
Homaro Cantu : And I'm Homaro .

DP : " English Country Garden . "

-------- 1.0000002
-------- 1.0000002
-------- 0.9342163
-------- 0.9283051
-------- 0.9283051
-------- 0.9206893
62669 8937 1.0000002 1.0 1.0
And it's very simple .

Actually it's very simple .

-------- 1.0000002
-------- 1.0000002
-------- 1.0000002
-------- 0.90760577
-------- 0.86564815
-------- 0.86564815
62688 62688 1.0000002 1.0 1.0
So how did that happen ?

So how did that happen ?

-------- 0.99999976
-------- 0.99999976
-------- 0.8114862
-------- 0.8013815
-------- 0.79608643
-------- 0.79357076
62701 75436 0.99999976 0.9999999 1.0
This got me thinking .

Which got me 

-------- 1.0
-------- 0.99104244
-------- 0.739511
-------- 0.7148806
-------- 0.7004257
-------- 0.6959009
70237 59780 0.99104244 0.99104255 0.9910423755645752
We changed the environment and we changed the behavior .

When we changed the environment , the behavior changed .

-------- 0.9999999
-------- 0.9999999
-------- 0.9999999
-------- 0.9488247
-------- 0.92754495
-------- 0.89883095
70238 70238 0.9999999 1.0 1.0
Here we have a problem .

Here we have a problem .

-------- 1.0
-------- 1.0
-------- 0.9107592
-------- 0.82092565
-------- 0.8016578
-------- 0.78615594
70343 70343 1.0 1.0 1.0
Close your eyes .

Close your eyes .

-------- 1.0000001
-------- 1.0000001
-------- 0.9165858
-------- 0.8848303
-------- 0.8703104
-------- 0.86554515
70349 70349 1.0000001 1.0 1.0
Now let me show you how this works .

Now let me show you how this works .

-------- 1.0000001
-------- 1.0000001
-------- 0.9925998
-------- 0.9925998
-------- 0.9925998
-------- 0.9925998
70550 70550 1.0000001 1.

-------- 1.0000001
-------- 1.0000001
-------- 1.0000001
-------- 1.0000001
-------- 1.0000001
-------- 1.0000001
78172 8147 1.0000001 1.0 1.0
HE : Thank you , Chad .

JE : Thank you , Tony .

-------- 1.0000001
-------- 0.99387443
-------- 0.98655725
-------- 0.985963
-------- 0.9631963
-------- 0.9631963
78254 63760 0.99387443 0.99387455 0.9938743710517883
All : A-one-two-three-four , one-two-three-four , one-two-three-four .

Audience : T , E , D .

-------- 0.9999999
-------- 0.9999999
-------- 0.94487774
-------- 0.9415166
-------- 0.91792065
-------- 0.9179204
78262 49006 0.9999999 0.99999994 1.0
So that's what I'm doing .

And that's what we're doing .

-------- 1.0000002
-------- 1.0000002
-------- 0.7308698
-------- 0.723983
-------- 0.7192147
-------- 0.69903064
78569 78569 1.0000002 1.0 1.0
We were always best in class .

We were always best in class .

-------- 1.0
-------- 1.0
-------- 1.0
-------- 1.0
-------- 1.0
-------- 0.9477682
78682 31286 1.0 0.99999994 1.0
And I sa

In [11]:
start = time.time()
saperate = " || "

src_label = "S"
emb_label = "E"
thres_arr = [0.8, 0.85, 0.9, 0.95]
for i in range(len(thres_arr)):
    threshold = thres_arr[i]
    out_file_write = open("en_train_EM_"+str(thres_arr[i]), 'w')
    out_file_write_score = open("en_train_EM_score_"+str(thres_arr[i]), 'w')
    out_file_write_factor = open("en_train_EM_factor_"+str(thres_arr[i]), 'w')
    num_gth_thres = 0

    for i in range(len(lst_srcfile)):
        j=5
        score = distance[i][j]
        content = ""
        content_label_arr = []
        if score >= threshold and score <0.999:
            num_gth_thres += 1
            best_simi_index = index[i][j]
            content = lst_srcfile[i].strip() + saperate + lst_tarfile[best_simi_index].strip()

            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label.append(emb_label)
            content_label = content_label + [emb_label] * len(lst_tarfile[best_simi_index].split())

            content_label_arr = " ".join(content_label)
        else:
            content = lst_srcfile[i].strip()
            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label_arr = " ".join(content_label)
 
        out_file_write.writelines(content+ "\n")    
        out_file_write_factor.writelines(content_label_arr + "\n")
        out_file_write_score.writelines(str(score) + "\n")

    print('threshold =', threshold, "EM sentences =", num_gth_thres, "/", len(lst_srcfile), num_gth_thres*100/len(lst_srcfile), "%" )

    out_file_write.close()
    out_file_write_score.close()
    out_file_write_factor.close()

done = time.time()
elapsed = done - start
print('=== Ending, Total time (second): ', elapsed)

threshold = 0.8 EM sentences = 4647 / 82899 5.605616473057576 %
threshold = 0.85 EM sentences = 2086 / 82899 2.516315033956984 %
threshold = 0.9 EM sentences = 785 / 82899 0.9469354274478582 %
threshold = 0.95 EM sentences = 199 / 82899 0.24005114657595386 %
=== Ending, Total time (second):  4.1623077392578125
